# Preparación de Entorno

In [1]:
# procesamiento_etl.py

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count, desc
from pyspark.sql.types import FloatType
import os

# --- 1. Preparar el entorno y cargar datos ---
print("==================================================")
print("PASO 1: PREPARAR ENTORNO Y CARGAR DATOS")
print("==================================================")

# Leer la URL del master desde la variable de entorno
# master_url = os.getenv("SPARK_MASTER_URL", "local[*]")

spark = SparkSession.builder \
    .appName("EcommerceAnalysis") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4041") \
    .getOrCreate()

spark.sparkContext.master
spark.sparkContext.uiWebUrl

print("Sesión de Spark creada exitosamente.")

# c. Cargar el archivo CSV en un DataFrame
# Asumimos que el archivo está en la ruta mapeada por el volumen
file_path = "/opt/spark/data/e-commerce_orders.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df = spark.read.option("delimiter", ";").csv(file_path, header=True, inferSchema=True)

print(f"Archivo '{file_path}' cargado.")
print(f"Esquema inferido:")
df.printSchema()
print("\nPrimeras 5 filas del DataFrame original:")
df.show(5)

PASO 1: PREPARAR ENTORNO Y CARGAR DATOS


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/15 01:39:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Sesión de Spark creada exitosamente.


Archivo '/opt/spark/data/e-commerce_orders.csv' cargado.
Esquema inferido:
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- shipping_charges: integer (nullable = true)
 |-- status: string (nullable = true)


Primeras 5 filas del DataFrame original:


+------------+------------+------------------------+-----------------+------+----------------+---------+
|    order_id| customer_id|order_purchase_timestamp|order_approved_at| price|shipping_charges|   status|
+------------+------------+------------------------+-----------------+------+----------------+---------+
|u6rPMRAYIGig|I74lXDOfoqsp|              18/11/2017|       18/11/2017|   241|             209|EJECUTADO|
|ohY8f4FEbX19|47TuLHF2s7X5|                2/6/2018|         2/6/2018|  4289|            1228|EJECUTADO|
|I28liQek73i2|dQ0dqI8Qwlj8|                8/1/2018|         9/1/2018|  5021|            6711|EJECUTADO|
|bBG1T89mlY8W|iQCmWhNkIczb|               10/3/2017|        10/3/2017|   891|            6205|EJECUTADO|
|CYxJJSQS8Lbo|Dp2g6JH8tO5Z|               2/12/2017|        5/12/2017|213999|             941|PENDIENTE|
+------------+------------+------------------------+-----------------+------+----------------+---------+
only showing top 5 rows


# Exploración y Limpieza

In [2]:
# --- 2. Exploración y limpieza ---
print("\n\n==================================================")
print("PASO 2: EXPLORACIÓN Y LIMPIEZA")
print("==================================================")

# a. Visualizar las primeras filas (ya hecho arriba, pero podemos repetirlo para seguir la estructura)
print("Visualizando las primeras filas para exploración inicial:")
df.show(5)



PASO 2: EXPLORACIÓN Y LIMPIEZA
Visualizando las primeras filas para exploración inicial:


[Stage 5:>                                                          (0 + 1) / 1]

+------------+------------+------------------------+-----------------+------+----------------+---------+
|    order_id| customer_id|order_purchase_timestamp|order_approved_at| price|shipping_charges|   status|
+------------+------------+------------------------+-----------------+------+----------------+---------+
|u6rPMRAYIGig|I74lXDOfoqsp|              18/11/2017|       18/11/2017|   241|             209|EJECUTADO|
|ohY8f4FEbX19|47TuLHF2s7X5|                2/6/2018|         2/6/2018|  4289|            1228|EJECUTADO|
|I28liQek73i2|dQ0dqI8Qwlj8|                8/1/2018|         9/1/2018|  5021|            6711|EJECUTADO|
|bBG1T89mlY8W|iQCmWhNkIczb|               10/3/2017|        10/3/2017|   891|            6205|EJECUTADO|
|CYxJJSQS8Lbo|Dp2g6JH8tO5Z|               2/12/2017|        5/12/2017|213999|             941|PENDIENTE|
+------------+------------+------------------------+-----------------+------+----------------+---------+
only showing top 5 rows


# validar y Eliminar Registros Nulos

In [3]:
# b. Validar y eliminar registros nulos
print("Contando registros nulos por columna antes de la limpieza:")
from pyspark.sql.functions import isnan, when, count
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

# Eliminar filas con cualquier valor nulo
df_clean = df.na.drop()
print(f"Registros antes de la limpieza: {df.count()}")
print(f"Registros después de eliminar nulos: {df_clean.count()}")

# c. Asegurar que los tipos de dato sean correctos
# Vamos a forzar el tipo de 'price' a FloatType para asegurar la precisión
df_clean = df_clean.withColumn("price", col("price").cast(FloatType()))
print("Asegurando que la columna 'price' sea de tipo FloatType.")
print("Nuevo esquema:")
df_clean.printSchema()

Contando registros nulos por columna antes de la limpieza:


+--------+-----------+------------------------+-----------------+-----+----------------+------+
|order_id|customer_id|order_purchase_timestamp|order_approved_at|price|shipping_charges|status|
+--------+-----------+------------------------+-----------------+-----+----------------+------+
|       0|          0|                       0|               36|    0|               0|     0|
+--------+-----------+------------------------+-----------------+-----+----------------+------+



Registros antes de la limpieza: 229674


Registros después de eliminar nulos: 229638
Asegurando que la columna 'price' sea de tipo FloatType.
Nuevo esquema:
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- price: float (nullable = true)
 |-- shipping_charges: integer (nullable = true)
 |-- status: string (nullable = true)



# Transformación y Metricas

In [4]:
# --- 3. Transformaciones y métricas ---
print("\n\n==================================================")
print("PASO 3: TRANSFORMACIONES Y MÉTRICAS")
print("==================================================")

# a. Calcular el monto total y promedio de ventas
sales_metrics_df = df_clean.agg(
    sum("price").alias("Monto_Total_Ventas"),
    avg("price").alias("Precio_Promedio_Venta")
)

# Mostrar en consola
sales_metrics_row = sales_metrics_df.collect()[0]
print("--- Métricas Generales de Ventas ---")
print(f"Monto Total de Ventas: {sales_metrics_row['Monto_Total_Ventas']:.2f}")
print(f"Precio Promedio de Venta: {sales_metrics_row['Precio_Promedio_Venta']:.2f}")


# b. Contar la cantidad de pedidos por estado
print("\n--- Cantidad de Pedidos por Estado ---")
orders_by_status = df_clean.groupBy("Status").count().orderBy(desc("count"))
orders_by_status.show()


# c. Obtener el top 3 de clientes que más dinero gastaron
print("\n--- Top 3 Clientes por Gasto Total ---")
top_customers = df_clean.groupBy("customer_id") \
    .agg(sum("price").alias("Total_Gastado")) \
    .orderBy(desc("Total_Gastado")) \
    .limit(3)
top_customers.show()

# Opcional: Guardar los resultados en CSV como ya lo hacías
# Esto demuestra la parte "Load" del ETL
print("\n\n==================================================")
print("PASO 4: CARGA DE RESULTADOS (LOAD)")
print("==================================================")
print("Guardando resultados en el directorio /opt/spark/data/resultados/...")



PASO 3: TRANSFORMACIONES Y MÉTRICAS


--- Métricas Generales de Ventas ---
Monto Total de Ventas: 3688375824.00
Precio Promedio de Venta: 16061.70

--- Cantidad de Pedidos por Estado ---


+----------+------+
|    Status| count|
+----------+------+
| EJECUTADO|198240|
| PENDIENTE| 27006|
|SUSPENDIDO|  4392|
+----------+------+


--- Top 3 Clientes por Gasto Total ---


[Stage 23:=============================>                            (1 + 1) / 2]

+------------+-------------+
| customer_id|Total_Gastado|
+------------+-------------+
|XF9C78TLFvuT|    2459994.0|
|pLsmwxaiSOTf|    2459994.0|
|v2q09pZEMFj4|    2459994.0|
+------------+-------------+



PASO 4: CARGA DE RESULTADOS (LOAD)
Guardando resultados en el directorio /opt/spark/data/resultados/...


In [5]:
# Escritura en CSV

sales_metrics_df.write.csv("/opt/spark/data/resultados/monto_promedio", mode="overwrite", header=True)
orders_by_status.write.csv("/opt/spark/data/resultados/pedidos_por_estado", mode="overwrite", header=True)
top_customers.write.csv("/opt/spark/data/resultados/top_clientes", mode="overwrite", header=True)
# Spark divide el dataset en particiones pero con coalesce reduce a una partición. No es recomendado en Spark porque los dataframes son generalmente cientos de veces mas grandes y afecta el rendimiento
df_clean.coalesce(1).write.csv("/opt/spark/data/resultados/limpio", mode="overwrite", header=True)

print("Proceso ETL completado exitosamente.")

25/10/15 01:43:20 ERROR FileFormatWriter: Aborting job c95c181b-7bbe-4995-a5b3-04a7e28f3179.
java.io.IOException: Failed to rename DeprecatedRawLocalFileStatus{path=file:/opt/spark/data/resultados/monto_promedio/_temporary/0/task_202510150143182481130857678437646_0026_m_000000/part-00000-eb6d83e5-0766-400d-b7b0-8fc783cd51df-c000.csv; isDirectory=false; length=74; replication=1; blocksize=33554432; modification_time=1760492599091; access_time=1760492599091; owner=; group=; permission=rw-rw-rw-; isSymlink=false; hasAcl=false; isEncrypted=false; isErasureCoded=false} to file:/opt/spark/data/resultados/monto_promedio/part-00000-eb6d83e5-0766-400d-b7b0-8fc783cd51df-c000.csv
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.mergePaths(FileOutputCommitter.java:477)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.mergePaths(FileOutputCommitter.java:490)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.commitJobInternal(FileOutputCommitter.java:405)
	at

Py4JJavaError: An error occurred while calling o163.csv.
: java.io.IOException: Failed to rename DeprecatedRawLocalFileStatus{path=file:/opt/spark/data/resultados/monto_promedio/_temporary/0/task_202510150143182481130857678437646_0026_m_000000/part-00000-eb6d83e5-0766-400d-b7b0-8fc783cd51df-c000.csv; isDirectory=false; length=74; replication=1; blocksize=33554432; modification_time=1760492599091; access_time=1760492599091; owner=; group=; permission=rw-rw-rw-; isSymlink=false; hasAcl=false; isEncrypted=false; isErasureCoded=false} to file:/opt/spark/data/resultados/monto_promedio/part-00000-eb6d83e5-0766-400d-b7b0-8fc783cd51df-c000.csv
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.mergePaths(FileOutputCommitter.java:477)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.mergePaths(FileOutputCommitter.java:490)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.commitJobInternal(FileOutputCommitter.java:405)
	at org.apache.hadoop.mapreduce.lib.o